In [ ]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

from qiskit_machine_learning.kernels import FidelityQuantumKernel
from qiskit.circuit.library import ZZFeatureMap
from qiskit.primitives import Sampler   # IMPORTANTE: reemplaza Aer para kernels

import numpy as np

# 1) Dataset
X, y = datasets.make_moons(n_samples=500, noise=0.2, random_state=42)

# Normalizar
X = StandardScaler().fit_transform(X)

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


# 2) Modelos con kernels clásicos
kernels = {
    "Lineal": SVC(kernel="linear"),
    "RBF": SVC(kernel="rbf", gamma='scale'),
    "Polynomial": SVC(kernel="poly", degree=3)
}

print("Entrenando modelos clásicos...\n")

for name, model in kernels.items():
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    acc = accuracy_score(y_test, preds)
    print(f"{name}: {acc:.3f}")


# 3) Quantum Kernel moderno
print("\nEntrenando modelo con Quantum Kernel...\n")

# FeatureMap cuántico (mismo usado en el artículo)
feature_map = ZZFeatureMap(feature_dimension=2, reps=2)

# Sampler moderno (reemplaza por completo a Aer)
sampler = Sampler()

quantum_kernel = FidelityQuantumKernel(
    feature_map=feature_map,
    sampler=sampler
)

# MATRIZ KERNEL (train y test)
K_train = quantum_kernel.evaluate(X_train)
K_test = quantum_kernel.evaluate(X_test, X_train)

# SVM con kernel precomputado
svc_q = SVC(kernel='precomputed')

svc_q.fit(K_train, y_train)
preds_q = svc_q.predict(K_test)
acc_q = accuracy_score(y_test, preds_q)

print(f"Quantum Kernel (ZZFeatureMap): {acc_q:.3f}")
